<a href="https://colab.research.google.com/github/Anna172/Mainor/blob/master/Domohoz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

In [0]:
ind = pd.read_csv("индивиды2.csv")
ind.head()

,ID_W,IDIND,ID_I,ID_H,STATUS,MARST,EDUC,H3,H4,AGE,J10,J57,J322
0,8,1,1010101,10101,3,2.0,19.0,1,1,25,294.0,NaN,NaN
1,12,1,1010101,10101,3,2.0,21.0,1,1,30,10700.0,NaN,NaN
2,26,1,1000101,10001,3,4.0,21.0,1,1,44,4000.0,NaN,4.0
3,6,1,1010101,10101,3,7.0,19.0,1,1,22,NaN,NaN,NaN
4,21,1,1000101,10001,3,5.0,21.0,1,1,39,43000.0,NaN,5.0


In [0]:
data = pd.read_csv("DomohChAll.csv")
data.head()

,ID_W,ID_H,STATUS,SITE,A3,A3_3,NFM,NFM1_O,NFM1_N,B1_1,IDIND1,B1_2_1,B1_2_2,B1_4,B1_5,B1_8,B2_1,IDIND2,B2_2_1,B2_2_2,B2_4,B2_5,B2_8,B2_9_1,B3_1,IDIND3,B3_2_1,B3_2_2,B3_4,B3_5,B3_8,B3_9_1,B3_9_2,B4_1,IDIND4,B4_2_1,B4_2_2,B4_4,B4_5,B4_8,...,H48_1,H48_2,H48_3,H48_4,H49_1,H49_2,H49_3,H49_4,H49_1_1,H49_1_2,H49_1_3,H49_1_4,H49_2_1,H49_2_2,H49_2_3,H49_2_4,H67,H68_1B,H68_2B,H68_3B,FATHER,MOTHER,CHILDS,YEAR,MARST_M,EDUC_M,H3_M,H4_M,J10_M,J57_M,J322_M,MARST_F,EDUC_F,H3_F,H4_F,AGE_F,J10_F,J57_F,J322_F,CHILDS_ALL
0,24,1170043,3,117,43,1.0,1,1.0,NaN,1.0,8281.0,1.0,NaN,2.0,1947.0,NaN,NaN,8282.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8283.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15836.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,27,580124,4,64,124,1.0,2,2.0,NaN,1.0,26180.0,1.0,NaN,1.0,1972.0,NaN,2.0,26181.0,1.0,NaN,2.0,1970.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,10,1000605,2,100,5,1.0,3,3.0,NaN,1.0,17382.0,1.0,NaN,2.0,1972.0,NaN,2.0,17383.0,1.0,NaN,1.0,1969.0,NaN,1.0,3.0,17384.0,1.0,NaN,1.0,1991.0,NaN,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,15,10004308,2,100,308,2.0,1,NaN,1.0,1.0,27123.0,NaN,NaN,1.0,1930.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0,2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,24,460027,1,46,27,1.0,1,1.0,NaN,1.0,33975.0,1.0,NaN,2.0,1947.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [0]:
data.shape


(131515, 353)

In [0]:
ind.shape

(353827, 13)

In [0]:
# Добавляем новые слобцы
data = data.assign(Childs_All=np.zeros(data.shape[0]))
#data = data.assign(Mother_Ed=np.zeros(data.shape[0]))
#data = data.assign(Childs_Ed=np.zeros(data.shape[0]))

In [0]:
data = data.assign(Baby1=np.zeros(data.shape[0]))
data = data.assign(Baby2=np.zeros(data.shape[0]))
data = data.assign(Baby3=np.zeros(data.shape[0]))
data = data.assign(Baby4=np.zeros(data.shape[0]))
data = data.assign(Baby5=np.zeros(data.shape[0]))
data = data.assign(Baby6=np.zeros(data.shape[0]))

In [0]:
data

,ID_W,ID_H,STATUS,SITE,A3,A3_3,NFM,NFM1_O,NFM1_N,B1_1,IDIND1,B1_2_1,B1_2_2,B1_4,B1_5,B1_8,B2_1,IDIND2,B2_2_1,B2_2_2,B2_4,B2_5,B2_8,B2_9_1,B3_1,IDIND3,B3_2_1,B3_2_2,B3_4,B3_5,B3_8,B3_9_1,B3_9_2,B4_1,IDIND4,B4_2_1,B4_2_2,B4_4,B4_5,B4_8,...,H48_1,H48_2,H48_3,H48_4,H49_1,H49_2,H49_3,H49_4,H49_1_1,H49_1_2,H49_1_3,H49_1_4,H49_2_1,H49_2_2,H49_2_3,H49_2_4,H67,H68_1B,H68_2B,H68_3B,FATHER,MOTHER,CHILDS,YEAR,MARST_M,EDUC_M,H3_M,H4_M,J10_M,J57_M,J322_M,MARST_F,EDUC_F,H3_F,H4_F,AGE_F,J10_F,J57_F,J322_F,Childs_All
0,23,1000213,4,103,213,1.0,4,4.0,NaN,1.0,29533.0,1.0,NaN,2.0,1976.0,NaN,NaN,29534.0,1.0,NaN,2.0,1994.0,NaN,4.0,3.0,29535.0,1.0,NaN,1.0,1998.0,NaN,4.0,6.0,4.0,46005.0,1.0,NaN,1.0,1978.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29534.0,29533,1,2014,3.0,17.0,213.0,1.0,10000.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,27,1290176,4,134,176,1.0,3,3.0,NaN,NaN,8739.0,2.0,NaN,NaN,NaN,NaN,2.0,8740.0,1.0,NaN,2.0,1965.0,NaN,NaN,3.0,8741.0,1.0,NaN,1.0,1965.0,NaN,NaN,1.0,NaN,8742.0,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,12,890153,2,89,53,1.0,3,3.0,NaN,NaN,5985.0,2.0,NaN,NaN,NaN,NaN,NaN,5986.0,2.0,NaN,NaN,NaN,NaN,NaN,3.0,5987.0,1.0,NaN,1.0,1980.0,NaN,NaN,NaN,NaN,5988.0,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0,2003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,16,3903013,3,39,13,1.0,1,1.0,NaN,NaN,20138.0,2.0,NaN,NaN,NaN,NaN,2.0,20139.0,1.0,NaN,1.0,1976.0,NaN,NaN,NaN,20140.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0,2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,24,1392117,1,138,2117,1.0,1,1.0,NaN,1.0,47386.0,1.0,NaN,2.0,1981.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131510,8,100902,2,10,2,1.0,5,5.0,NaN,1.0,674.0,1.0,NaN,2.0,1956.0,NaN,2.0,675.0,1.0,NaN,1.0,1954.0,NaN,1.0,3.0,676.0,1.0,NaN,2.0,1977.0,NaN,4.0,4.0,4.0,677.0,1.0,NaN,1.0,1979.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,675.0,674,1,1998,2.0,14.0,2.0,1.0,188.0,NaN,NaN,2.0,14.0,2.0,2.0,44.0,100.0,NaN,NaN,0.0
131511,24,1383952,1,138,3952,1.0,4,4.0,NaN,1.0,19548.0,1.0,NaN,2.0,1939.0,NaN,2.0,19549.0,1.0,NaN,2.0,1975.0,NaN,4.0,3.0,21207.0,1.0,NaN,1.0,1963.0,NaN,4.0,6.0,NaN,21208.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19549.0,19548,2,2015,5.0,14.0,3952.0,1.0,NaN,NaN,5.0,4.0,21.0,3952.0,2.0,40.0,35000.0,NaN,4.0,0.0
131512,14,30002,4,3,2,1.0,6,6.0,NaN,1.0,92.0,1.0,NaN,2.0,1958.0,NaN,2.0,93.0,1.0,NaN,1.0,1954.0,NaN,1.0,NaN,94.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,95.0,1.0,NaN,2.0,1927.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93.0,92,1,2005,2.0,16.0,2.0,1.0,6000.0,NaN,NaN,2.0,14.0,2.0,2.0,51.0,6000.0,NaN,NaN,0.0
131513,14,660608,1,66,8,1.0,4,4.0,NaN,1.0,14922.0,1.0,NaN,2.0,1952.0,NaN,2.0,14923.0,1.0,NaN,1.0,1952.0,NaN,1.0,3.0,14924.0,1.0,NaN,1.0,1980.0,NaN,4.0,4.0,4.0,14925.0,1.0,NaN,1.0,1985.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14923.0,14922,2,2005,2.0,21.0,8.0,1.0,2000.0,NaN,NaN,2.0,13.0,8.0,2.0,53.0,10000.0,NaN,NaN,0.0


In [0]:
def get_curr_year(value):
    if value >= 9:
        return 1991 + value
    elif value <= 7:
        return 1989 + value
    elif value == 8:
        return 1998

In [0]:
class Member:
    def __init__(self, index):
        self.index = index
        self.gender = None
        self.mother = -1
        self.father = -1
        self.childs = []

In [0]:
from operator import attrgetter
member_id = ['IDIND' + str(v) for v in range(1, 25)]

In [0]:
for main_index in range(len(data)):
# for main_index in range(2):
    
    number_members = int(data.loc[main_index, 'NFM']) # Количество членов семьи
    members_data = []
 
    # Парсим по IDIND@
    for i in range(number_members):
        member = data.loc[main_index, member_id[i]:member_id[i+1]]
        members_data.append(member)

    # Инициализируем классы членов семьи
    members = [Member(i + 1) for i in range(number_members)]
    for i in range(2, number_members + 1):
        for j in range(i-1, 0, -1):
            #Ячейка  B@.9.@, при разных варианциях имеет разные форматы
            if (i >= 11 and i <= 16) and (j >= 10):
                cell = "B" + str(i) + "9_" + str(j)
            elif (i >= 17 and i <= 24) and (j >= 10):
                cell = "B" + str(i) + "_9" + str(j)
            else:
                cell = "B" + str(i) + "_9_" + str(j)
            
            #Если член семьи является отцом/матерью
            if data.loc[main_index, cell] == 2:
                members[i-1].childs.append(j-1)
                gender = "B" + str(i) + "_4"
                members[i-1].gender = data.loc[main_index, gender] # Присваиваем пол
                
                # В зависимости от пола родителя присваиваем соответствующее значение
                if data.loc[main_index, gender] == 1:
                    members[j-1].father = i-1
                elif data.loc[main_index, gender] == 2:
                    members[j-1].mother = i-1  
                    
            #Если член семьи является сыном/дочерью       
            elif data.loc[main_index, cell] == 4:
                members[j-1].childs.append(i-1)
                gender = "B" + str(j) + "_4"
                members[j-1].gender = data.loc[main_index, gender] # Присваиваем пол
                
                # В зависимости от пола родителя присваиваем соответствующее значение
                members[i-1].father = j
                if data.loc[main_index, gender] == 1:
                    members[i-1].father = j-1
                elif data.loc[main_index, gender] == 2:
                    members[i-1].mother = j-1
       
#     for mem in members:
#         print(mem.index, mem.father, mem.mother, mem.childs)
        
    # Выбираем детей старше 19 лет
    childs = []
    all_childs = []
    for i in range(number_members):
        if not members[i].childs and (members[i].father != -1 or members[i].mother != -1):
            year = "B" + str(i + 1) + "_5"
            all_childs.append(members[i])
            if data.loc[main_index, year] != None and data.loc[main_index, 'ID_W'] != None \
            and (get_curr_year(data.loc[main_index, 'ID_W']) - data.loc[main_index, year] > 19):
                childs.append(members[i])
    
    if not childs:
        continue
    if not all_childs:
      continue
    
    # Если получилось так, что есть несколько групп родителей-детей, то выбираем более многочисленную
    sorted_childs = sorted(childs, key=attrgetter('mother', 'father'))
    mother, father = sorted_childs[0].mother, sorted_childs[0].father
    sorted_all_childs = sorted(all_childs, key=attrgetter('mother', 'father'))
    
    last_group, curr_group = 0, 1
    index = 0
    
    for i in range(1, len(sorted_childs)):
        if sorted_childs[i].mother != mother or sorted_childs[i].father != father:
            if curr_group > last_group:   
                last_group = curr_group
                curr_group = 1
                index = i - 1
        else:
            curr_group += 1
    
    if curr_group > last_group:   
        last_group = curr_group
        curr_group = 1
        index = len(sorted_childs) - 1
    
    # last_group - самая большая группа родителей-детей
    # Инициализируем новые столбцы
    
    mother_index = sorted_childs[index].mother
    father_index = sorted_childs[index].father
    
    all_childs_group = 0

    for k in range(len(sorted_all_childs)):
        if sorted_all_childs[k].mother == mother_index or sorted_all_childs[k].father == father_index:
            all_childs_group += 1
    
#     print(mother_index, father_index)
    
    #if mother_index != -1:
    #    mother_ID = "IDIND" + str(mother_index + 1)
    #    data.loc[main_index, 'Mother'] = data.loc[main_index, mother_ID]
    #if father_index != -1: 
    #    father_ID = "IDIND" + str(father_index + 1)
    #    data.loc[main_index, 'Father'] = data.loc[main_index, father_ID]
    #data.loc[main_index, 'Childs'] = last_group
    
    data.loc[main_index, 'Childs_All'] = all_childs_group
    

In [0]:
data.head()

,ID_W,ID_H,STATUS,SITE,A3,A3_3,NFM,NFM1_O,NFM1_N,B1_1,IDIND1,B1_2_1,B1_2_2,B1_4,B1_5,B1_8,B2_1,IDIND2,B2_2_1,B2_2_2,B2_4,B2_5,B2_8,B2_9_1,B3_1,IDIND3,B3_2_1,B3_2_2,B3_4,B3_5,B3_8,B3_9_1,B3_9_2,B4_1,IDIND4,B4_2_1,B4_2_2,B4_4,B4_5,B4_8,...,H48_4,H49_1,H49_2,H49_3,H49_4,H49_1_1,H49_1_2,H49_1_3,H49_1_4,H49_2_1,H49_2_2,H49_2_3,H49_2_4,H67,H68_1B,H68_2B,H68_3B,FATHER,MOTHER,CHILDS,YEAR,MARST_M,EDUC_M,H3_M,H4_M,J10_M,J57_M,J322_M,MARST_F,EDUC_F,H3_F,H4_F,AGE_F,J10_F,J57_F,J322_F,Childs_All,Mother,Father,Childs
0,23,1000213,4,103,213,1.0,4,4.0,NaN,1.0,29533.0,1.0,NaN,2.0,1976.0,NaN,NaN,29534.0,1.0,NaN,2.0,1994.0,NaN,4.0,3.0,29535.0,1.0,NaN,1.0,1998.0,NaN,4.0,6.0,4.0,46005.0,1.0,NaN,1.0,1978.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29534.0,29533,1,2014,3.0,17.0,213.0,1.0,10000.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,29533.0,29534.0,1.0
1,27,1290176,4,134,176,1.0,3,3.0,NaN,NaN,8739.0,2.0,NaN,NaN,NaN,NaN,2.0,8740.0,1.0,NaN,2.0,1965.0,NaN,NaN,3.0,8741.0,1.0,NaN,1.0,1965.0,NaN,NaN,1.0,NaN,8742.0,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
2,12,890153,2,89,53,1.0,3,3.0,NaN,NaN,5985.0,2.0,NaN,NaN,NaN,NaN,NaN,5986.0,2.0,NaN,NaN,NaN,NaN,NaN,3.0,5987.0,1.0,NaN,1.0,1980.0,NaN,NaN,NaN,NaN,5988.0,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0,2003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
3,16,3903013,3,39,13,1.0,1,1.0,NaN,NaN,20138.0,2.0,NaN,NaN,NaN,NaN,2.0,20139.0,1.0,NaN,1.0,1976.0,NaN,NaN,NaN,20140.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0,2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
4,24,1392117,1,138,2117,1.0,1,1.0,NaN,1.0,47386.0,1.0,NaN,2.0,1981.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN


In [0]:
# Добавляем столбец доходов
data = data.assign(Income_M=np.zeros(data.shape[0]))
data = data.assign(Income_F=np.zeros(data.shape[0]))
#data = data.assign(vspom_M=np.zeros(data.shape[0]))
#data = data.assign(vspom_F=np.zeros(data.shape[0]))

In [0]:
type(data['J10_M'])

NameError: ignored

In [0]:
#data['vspom_F'] = data['J57_F']
#data['J10_M'].fillna(0, inplace=True)
#data['vspom_F'].fillna(0, inplace=True)
data['J10_M'].fillna(0, inplace=True)
data['J10_F'].fillna(0, inplace=True)
data['J57_M'].fillna(0, inplace=True)
data['J57_F'].fillna(0, inplace=True)

data['Income_M'] = data['J10_M'] + data['J57_M']
data['Income_F'] = data['J10_F'] + data['J57_F']
data['Income_M'].replace(0, np.nan, inplace=True)
data['Income_F'].replace(0, np.nan, inplace=True)

data['J10_M'].replace(0, np.nan, inplace=True)
data['J10_F'].replace(0, np.nan, inplace=True)
data['J57_M'].replace(0, np.nan, inplace=True)
data['J57_F'].replace(0, np.nan, inplace=True)

In [0]:
data.head()

,ID_W,ID_H,STATUS,SITE,A3,A3_3,NFM,NFM1_O,NFM1_N,B1_1,IDIND1,B1_2_1,B1_2_2,B1_4,B1_5,B1_8,B2_1,IDIND2,B2_2_1,B2_2_2,B2_4,B2_5,B2_8,B2_9_1,B3_1,IDIND3,B3_2_1,B3_2_2,B3_4,B3_5,B3_8,B3_9_1,B3_9_2,B4_1,IDIND4,B4_2_1,B4_2_2,B4_4,B4_5,B4_8,...,H48_3,H48_4,H49_1,H49_2,H49_3,H49_4,H49_1_1,H49_1_2,H49_1_3,H49_1_4,H49_2_1,H49_2_2,H49_2_3,H49_2_4,H67,H68_1B,H68_2B,H68_3B,FATHER,MOTHER,CHILDS,YEAR,MARST_M,EDUC_M,H3_M,H4_M,J10_M,J57_M,J322_M,MARST_F,EDUC_F,H3_F,H4_F,AGE_F,J10_F,J57_F,J322_F,CHILDS_ALL,Income_M,Income_F
0,24,1170043,3,117,43,1.0,1,1.0,NaN,1.0,8281.0,1.0,NaN,2.0,1947.0,NaN,NaN,8282.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8283.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15836.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
1,27,580124,4,64,124,1.0,2,2.0,NaN,1.0,26180.0,1.0,NaN,1.0,1972.0,NaN,2.0,26181.0,1.0,NaN,2.0,1970.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
2,10,1000605,2,100,5,1.0,3,3.0,NaN,1.0,17382.0,1.0,NaN,2.0,1972.0,NaN,2.0,17383.0,1.0,NaN,1.0,1969.0,NaN,1.0,3.0,17384.0,1.0,NaN,1.0,1991.0,NaN,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0,2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
3,15,10004308,2,100,308,2.0,1,NaN,1.0,1.0,27123.0,NaN,NaN,1.0,1930.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0,2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
4,24,460027,1,46,27,1.0,1,1.0,NaN,1.0,33975.0,1.0,NaN,2.0,1947.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN


In [0]:
year['year']

0         1994
1         1994
2         1994
3         1994
4         1994
          ... 
131510    2018
131511    2018
131512    2018
131513    2018
131514    2018
Name: year, Length: 131515, dtype: int64

In [0]:
data['year'] = year['year']

In [0]:
# Добавляем новые слобцы
data = data.assign(Father=np.zeros(data.shape[0]))
data = data.assign(Mother=np.zeros(data.shape[0]))
data = data.assign(Childs=np.zeros(data.shape[0]))

In [0]:
x = []
for i in range(131515):
  if int(data.loc[i, 'year']) == 1995 and int(data.loc[i,'ID_W']) !=6:
    x.append(int(data.loc[i, 'year']))


In [0]:
print(x)

[]


In [0]:
data.shape[0]

131515

In [0]:
data

,ID_W,ID_H,STATUS,SITE,A3,A3_3,NFM,NFM1_O,NFM1_N,B1_1,IDIND1,B1_2_1,B1_2_2,B1_1_23,B1_2_23,B1_3,B1_4,B1_5,B1_8,B2_1,IDIND2,B2_2_1,B2_2_2,B2_1_23,B2_2_23,B2_3,B2_4,B2_5,B2_8,B2_9_1,B3_1,IDIND3,B3_2_1,B3_2_2,B3_1_23,B3_2_23,B3_3,B3_4,B3_5,B3_8,...,B19_912,B19_913,B19_914,B19_915,B19_916,B19_917,B19_918,H45,H46_1B,H46_2B,H46_3B,H46_4B,H461_1,H461_2,H461_3,H461_4,H48_1,H48_2,H48_3,H48_4,H49_1,H49_2,H49_3,H49_4,H49_1_1,H49_1_2,H49_1_3,H49_1_4,H49_2_1,H49_2_2,H49_2_3,H49_2_4,H67,H68_1B,H68_2B,H68_3B,Father,Mother,Childs,year
0,5,1001,3,1,1,NaN,2,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,11.0,2.0,1973.0,2.0,2.0,2.0,NaN,NaN,NaN,NaN,11.0,1.0,1971.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1994
1,5,1002,3,1,2,NaN,4,NaN,NaN,1.0,3.0,NaN,NaN,NaN,NaN,12.0,2.0,1955.0,3.0,2.0,4.0,NaN,NaN,NaN,NaN,12.0,1.0,1949.0,3.0,1.0,3.0,5.0,NaN,NaN,NaN,NaN,12.0,2.0,1984.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1994
2,5,1003,3,1,3,NaN,1,NaN,NaN,1.0,7.0,NaN,NaN,NaN,NaN,12.0,2.0,1944.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1994
3,5,1004,3,1,4,NaN,1,NaN,NaN,1.0,8.0,NaN,NaN,NaN,NaN,12.0,2.0,1968.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1994
4,5,1005,3,1,5,NaN,2,NaN,NaN,1.0,9.0,NaN,NaN,NaN,NaN,12.0,2.0,1946.0,2.0,2.0,10.0,NaN,NaN,NaN,NaN,12.0,1.0,1951.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131510,27,14215911,3,189,15911,2.0,4,NaN,4.0,1.0,58373.0,NaN,NaN,NaN,NaN,12.0,2.0,1983.0,NaN,NaN,58481.0,NaN,NaN,NaN,NaN,12.0,1.0,1975.0,NaN,1.0,3.0,58374.0,NaN,NaN,NaN,NaN,12.0,2.0,2014.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,2018
131511,27,14215933,3,189,15933,2.0,2,NaN,2.0,1.0,58376.0,NaN,NaN,NaN,NaN,12.0,2.0,1936.0,NaN,2.0,58377.0,NaN,NaN,NaN,NaN,12.0,1.0,1933.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,2018
131512,27,14216025,3,189,16025,2.0,2,NaN,2.0,1.0,58378.0,NaN,NaN,NaN,NaN,12.0,2.0,1966.0,NaN,2.0,58379.0,NaN,NaN,NaN,NaN,12.0,2.0,1987.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58379.0,58378.0,1.0,2018
131513,27,14216456,3,150,16456,2.0,2,NaN,2.0,1.0,58380.0,NaN,NaN,NaN,NaN,12.0,1.0,1979.0,NaN,NaN,58482.0,NaN,NaN,NaN,NaN,12.0,2.0,1982.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,2018


In [0]:
def get_curr_year(value):
    if value >= 9:
        return 1991 + value
    elif value <= 7:
        return 1989 + value
    elif value == 8:
        return 1998

In [0]:
class Member:
    def __init__(self, index):
        self.index = index
        self.gender = None
        self.mother = -1
        self.father = -1
        self.childs = []

In [0]:
from operator import attrgetter
member_id = ['IDIND' + str(v) for v in range(1, 25)]

In [0]:
for main_index in range(len(data)):
# for main_index in range(2):
    
    number_members = int(data.loc[main_index, 'NFM']) # Количество членов семьи
    members_data = []
 
    # Парсим по IDIND@
    for i in range(number_members):
        member = data.loc[main_index, member_id[i]:member_id[i+1]]
        members_data.append(member)

    # Инициализируем классы членов семьи
    members = [Member(i + 1) for i in range(number_members)]
    for i in range(2, number_members + 1):
        for j in range(i-1, 0, -1):
            #Ячейка  B@.9.@, при разных варианциях имеет разные форматы
            if (i >= 11 and i <= 16) and (j >= 10):
                cell = "B" + str(i) + "9_" + str(j)
            elif (i >= 17 and i <= 24) and (j >= 10):
                cell = "B" + str(i) + "_9" + str(j)
            else:
                cell = "B" + str(i) + "_9_" + str(j)
            
            #Если член семьи является отцом/матерью
            if data.loc[main_index, cell] == 2:
                members[i-1].childs.append(j-1)
                gender = "B" + str(i) + "_4"
                members[i-1].gender = data.loc[main_index, gender] # Присваиваем пол
                
                # В зависимости от пола родителя присваиваем соответствующее значение
                if data.loc[main_index, gender] == 1:
                    members[j-1].father = i-1
                elif data.loc[main_index, gender] == 2:
                    members[j-1].mother = i-1  
                    
            #Если член семьи является сыном/дочерью       
            elif data.loc[main_index, cell] == 4:
                members[j-1].childs.append(i-1)
                gender = "B" + str(j) + "_4"
                members[j-1].gender = data.loc[main_index, gender] # Присваиваем пол
                
                # В зависимости от пола родителя присваиваем соответствующее значение
                members[i-1].father = j
                if data.loc[main_index, gender] == 1:
                    members[i-1].father = j-1
                elif data.loc[main_index, gender] == 2:
                    members[i-1].mother = j-1
       
#     for mem in members:
#         print(mem.index, mem.father, mem.mother, mem.childs)
        
    # Выбираем детей старше 19 лет
    childs = []
    for i in range(number_members):
        if not members[i].childs and (members[i].father != -1 or members[i].mother != -1):
            year = "B" + str(i + 1) + "_5"
            if data.loc[main_index, year] != None and data.loc[main_index, 'ID_W'] != None \
            and (get_curr_year(data.loc[main_index, 'ID_W']) - data.loc[main_index, year] > 19):
                childs.append(members[i])
    
    if not childs:
        continue
    
    # Если получилось так, что есть несколько групп родителей-детей, то выбираем более многочисленную
    sorted_childs = sorted(childs, key=attrgetter('mother', 'father'))
    mother, father = sorted_childs[0].mother, sorted_childs[0].father
    
    last_group, curr_group = 0, 1
    index = 0
    
    for i in range(1, len(sorted_childs)):
        if sorted_childs[i].mother != mother or sorted_childs[i].father != father:
            if curr_group > last_group:   
                last_group = curr_group
                curr_group = 1
                index = i - 1
        else:
            curr_group += 1
    
    if curr_group > last_group:   
        last_group = curr_group
        curr_group = 1
        index = len(sorted_childs) - 1
    
    # last_group - самая большая группа родителей-детей
    # Инициализируем новые столбцы
    
    mother_index = sorted_childs[index].mother
    father_index = sorted_childs[index].father
    
#     print(mother_index, father_index)
    
    if mother_index != -1:
        mother_ID = "IDIND" + str(mother_index + 1)
        data.loc[main_index, 'Mother'] = data.loc[main_index, mother_ID]
    if father_index != -1: 
        father_ID = "IDIND" + str(father_index + 1)
        data.loc[main_index, 'Father'] = data.loc[main_index, father_ID]
    data.loc[main_index, 'Childs'] = last_group
    

1 -1
0 1
1 -1
0 1
0 1
1 -1
2 -1
0 1
0 1
0 1
0 1
0 1
2 -1
1 -1
1 0
0 1
1 0
-1 2
0 1
-1 1
0 1
0 1
1 0
1 0
0 1
0 1
1 0
0 1
1 0
0 1
0 1
0 1
-1 2
1 0
0 1
0 1
0 1
0 1
0 1
0 1
2 -1
0 1
2 -1
0 1
0 1
0 1
0 1
1 0
1 0
1 -1
0 1
0 1
0 1
1 0
0 1
0 1
2 -1
3 -1
0 1
2 -1
2 -1
1 -1
0 1
0 1
1 0
1 0
2 -1
0 1
1 -1
2 -1
0 1
1 2
0 1
1 -1
1 -1
0 1
0 1
0 1
1 2
0 1
1 0
2 -1
0 1
2 -1
0 1
2 -1
1 0
1 0
0 1
1 0
2 -1
3 2
0 1
-1 0
0 1
1 0
-1 1
1 -1
0 1
0 1
-1 0
1 -1
0 1
0 1
0 1
0 1
0 1
2 -1
0 1
2 1
1 0
2 1
0 1
0 1
0 1
0 1
1 -1
2 -1
1 -1
0 1
0 1
1 2
1 0
-1 1
0 1
1 -1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
1 0
0 1
0 1
7 0
1 -1
2 1
0 1
0 1
0 1
1 0
3 -1
2 -1
0 1
1 -1
0 1
1 0
0 1
0 1
0 1
2 0
0 1
0 1
3 2
2 -1
0 1
0 1
0 1
1 0
0 1
0 1
0 1
2 -1
2 -1
2 -1
1 0
0 1
0 1
0 1
1 0
0 1
1 0
0 1
1 0
0 1
0 1
-1 0
1 2
-1 1
0 1
0 1
1 -1
0 1
0 1
1 0
0 1
0 1
0 1
1 -1
1 0
0 1
2 -1
0 1
0 1
1 0
0 1
0 1
1 2
0 1
2 -1
0 1
2 1
2 -1
0 1
0 2
-1 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
2 3
1 -1
1 0
1 -1
1 0
1 0
1 -1
0 1
0 1
1 -1
-1 1
1 0
0 1
0 1
0 1
0 1
2 1
1 2
1 

1 0
0 2
0 1
0 1
2 -1
0 1
0 1
0 1
1 -1
0 1
0 1
1 0
1 2
2 1
2 -1
0 1
2 1
1 -1
0 2
0 1
0 1
0 1
0 1
0 1
0 1
0 1
1 0
0 1
1 2
1 0
0 1
0 1
0 1
0 1
0 1
1 0
0 1
1 2
0 1
0 1
1 0
1 0
1 0
0 1
2 -1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
1 0
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
1 0
0 1
1 0
0 1
0 1
0 1
4 -1
0 1
1 0
0 1
0 1
0 1
2 -1
0 1
3 -1
0 1
0 1
0 1
0 1
0 1
0 1
1 0
0 1
0 1
1 0
0 1
0 1
0 1
0 1
0 1
0 1
0 1
2 -1
2 -1
0 1
1 0
2 -1
0 1
1 -1
0 1
0 1
0 1
1 -1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
1 2
0 1
0 1
1 0
1 2
2 -1
0 1
0 1
0 1
2 -1
1 0
-1 0
1 0
1 0
1 0
0 1
1 0
0 1
2 -1
3 2
0 1
0 1
0 1
-1 1
1 0
0 1
0 1
0 1
0 1
1 0
-1 0
1 -1
0 1
0 1
2 -1
0 1
0 1
2 1
0 1
0 1
0 1
0 1
0 1
1 0
1 -1
1 0
2 -1
0 1
3 0
0 1
0 1
0 1
1 0
1 0
-1 0
0 1
0 1
1 -1
0 1
0 1
1 0
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
1 0
0 1
0 1
0 1
1 0
1 0
0 1
0 1
0 1
0 1
-1 0
0 1
2 -1
0 1
0 1
0 1
0 1
1 0
0 1
0 1
0 1
0 1
0 1
0 1
2 0
0 1
2 -1
1 2
0 1
0 1
0 1
0 1
0 1
2 -1
0 1
-1 3
0 1
0 1
0 1
1 2
0 1
1 2
-1 2
0 1
0 1
0 1
0 1
-1 1
0 1
-1 2
0 1
1 0
0 1
0 1
0 1
1 0
0 1
2 -

0 1
-1 0
1 -1
0 1
0 1
1 -1
0 1
0 1
2 -1
3 4
1 0
0 1
1 2
0 1
1 0
0 1
0 1
2 -1
1 2
0 1
4 5
0 3
2 3
0 1
1 0
0 1
1 2
1 2
0 1
1 2
0 1
0 1
2 1
0 1
0 1
1 -1
0 1
0 1
0 1
0 1
1 -1
1 -1
0 1
0 1
0 1
1 0
1 0
0 1
1 0
2 1
0 1
1 2
0 1
0 1
0 1
0 1
0 1
0 1
1 -1
0 1
1 -1
1 0
0 1
0 1
0 1
0 1
1 -1
0 2
0 1
0 1
1 2
1 0
2 -1
1 2
0 1
0 1
0 1
1 0
1 0
1 0
0 1
1 -1
0 1
0 1
0 1
2 -1
0 1
0 1
0 1
0 1
0 1
1 2
1 0
1 0
0 1
0 2
1 2
0 1
0 1
0 1
0 1
0 1
0 1
2 -1
1 -1
0 1
1 -1
1 -1
0 1
1 -1
1 0
0 1
1 -1
0 1
1 2
0 1
0 1
0 1
0 1
0 1
0 1
2 -1
0 1
0 1
0 1
0 1
-1 1
2 -1
0 1
0 1
1 -1
1 -1
-1 1
0 1
1 2
0 1
0 1
0 1
0 1
1 2
1 2
-1 1
0 1
0 1
0 1
0 1
0 1
0 1
1 -1
0 1
0 1
0 1
0 1
1 -1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
1 -1
0 1
3 2
0 1
0 1
0 1
-1 1
0 1
0 1
0 1
0 1
0 3
0 1
0 1
0 1
2 3
0 1
0 1
0 1
0 1
1 2
0 1
0 1
-1 0
0 1
1 -1
0 1
0 1
0 1
2 3
1 -1
1 0
0 1
1 0
2 -1
-1 0
2 3
0 1
0 1
0 1
0 1
0 1
1 0
0 1
1 2
0 1
0 1
0 1
0 1
0 1
0 1
0 1
1 2
0 1
0 1
0 1
1 0
1 3
0 1
0 1
0 1
0 1
0 1
2 3
0 1
0 1
0 1
1 0
1 0
0 1
0 1
1 2
0 1
1 0
1 2
1 0
1 0
2 1
0 1


3 -1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
2 3
1 0
0 1
0 1
0 1
0 1
1 2
1 2
2 0
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
1 0
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 2
0 1
0 1
0 1
1 2
2 3
0 1
1 0
0 1
0 1
1 0
0 1
0 1
0 1
0 1
1 0
0 1
0 1
3 4
0 1
0 1
1 -1
0 1
1 2
0 1
0 1
2 -1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
1 0
0 1
0 1
1 2
0 1
1 0
0 1
1 0
0 1
0 1
1 2
0 1
0 1
1 2
1 0
-1 1
0 1
0 1
0 1
0 1
1 -1
0 1
1 0
0 1
0 1
0 1
0 1
0 1
0 1
1 0
1 0
0 1
0 1
0 1
0 1
-1 4
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
1 0
0 1
1 0
0 1
1 0
1 0
1 0
1 0
0 1
0 1
0 1
1 0
2 1
1 0
0 1
1 0
1 0
-1 0
0 1
1 0
0 1
1 0
0 1
7 -1
4 5
1 0
1 0
1 0
1 0
5 1
1 2
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
0 1
1 2
1 2
4 5
1 0
1 0
1 0
1 0
2 1
1 0
1 2
0 1
0 1
1 -1
0 1
1 2
0 1
0 1
1 2
0 1
0 1
0 1
0 1
1 0
0 1
1 0
0 1
1 0
0 1
0 1
0 1
0 1
0 1
0 1
0 1
1 -1
2 4
0 1
0 1
2 -1
2 1
0 1
0 1
0 1
0 1
2 3
0 1
0 1
1 0
0 1
-1 0
-1 0
1 0
2 -1
0 1
0 1
0 1
1 0
0 1
-1 1
0 1
0 1
-1 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
1 0
0 1
0 1
0 1
0 1
0 1
0 1
0 1
1 2
0 1
1 2
0 1
0 1


0 1
1 0
0 1
0 1
0 1
0 1
0 1
1 2
0 1
0 1
0 1
0 1
0 1
1 2
0 1
1 0
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
2 3
0 1
1 2
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
1 2
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
-1 0
0 1
1 0
0 1
1 0
0 1
0 1
2 -1
0 1
1 2
2 1
0 1
1 0
-1 1
0 1
-1 1
0 1
1 0
1 2
1 0
1 0
0 1
0 1
1 0
-1 1
0 1
0 1
0 1
0 1
0 1
1 2
3 4
1 0
0 1
1 0
1 0
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
1 0
1 0
1 0
0 2
0 1
1 2
0 1
0 1
0 1
0 1
0 3
0 1
0 1
0 1
1 0
1 2
-1 2
0 1
0 1
0 1
1 0
2 -1
1 0
0 1
0 1
0 1
0 1
0 1
0 1
2 3
2 -1
0 1
0 1
0 1
1 0
0 1
0 1
1 0
1 0
1 -1
1 0
0 1
0 1
1 0
0 1
1 0
0 1
1 0
1 0
0 1
1 0
0 1
0 1
0 1
5 4
1 2
0 1
0 1
1 -1
0 1
0 1
1 0
2 3
3 2
2 1
0 1
0 1
0 1
0 1
0 1
1 2
0 1
0 1
0 1
1 0
-1 1
0 1
1 0
0 1
0 1
0 1
1 2
0 1
0 1
0 1
0 1
0 1
0 1
0 2
0 1
0 1
2 -1
0 1
0 1
0 1
1 0
0 1
0 1
0 1
0 1
-1 1
0 1
1 0
0 1
1 0
0 1
0 2
-1 1
0 1
1 2
0 1
0 1
0 1
0 1
0 1
1 0
0 1
0 1
0 1
0 1
2 -1
0 1
2 3
0 1
0 1
0 1
0 1
2 -1
0 1
0 1
0 1
2 3
0 1
0 1
0 1
1 0
1 0
0 1
-1 1
0 1
-1 1
0 1
0 1
1 -1
0 1
0 2
0 1
-

...

0 1
0 1
0 1
0 1
0 1
-1 1
1 0
2 -1
1 2
0 1
0 1
0 1
0 1
4 5
0 1
0 1
-1 1
2 3
3 4
0 1
0 1
0 1
1 0
1 -1
0 1
1 0
0 1
0 1
4 3
1 0
0 1
2 3
1 2
0 1
2 1
0 1
0 1
0 1
0 1
0 1
2 -1
-1 1
0 1
0 1
5 -1
1 2
0 1
2 3
0 1
0 1
0 1
1 0
-1 1
1 0
0 1
0 1
0 1
1 2
1 2
3 2
0 1
-1 2
0 1
0 1
0 1
0 1
0 1
0 1
0 1
-1 1
2 -1
1 0
0 1
0 1
1 0
1 0
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
1 0
0 1
1 2
-1 1
0 1
0 1
-1 1
2 1
1 -1
0 1
0 1
0 1
0 1
0 1
0 1
1 2
2 -1
0 1
0 1
0 1
1 2
2 3
0 1
0 1
0 1
-1 1
0 1
1 2
0 1
0 1
0 1
1 -1
2 -1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
2 3
0 1
0 1
2 1
0 1
0 1
0 1
1 2
2 3
1 0
1 -1
0 1
2 3
1 2
0 1
-1 2
1 -1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
1 -1
0 1
3 4
0 1
-1 0
0 1
0 1
2 3
0 1
0 1
0 1
0 1
0 1
-1 1
2 3
0 1
0 1
1 -1
1 0
1 0
0 1
0 1
0 1
0 1
2 3
-1 0
0 1
3 4
0 1
1 -1
0 1
0 1
0 1
0 1
-1 0
1 -1
3 2
0 1
1 2
0 1
1 2
0 1
1 0
0 1
1 0
0 1
0 1
-1 1
0 1
0 1
0 1
3 0
1 -1
0 1
-1 0
0 1
0 1
1 -1
0 1
0 1
0 1
0 1
0 1
0 1
1 0
1 0
-1 0
-1 1
1 2
0 1
0 1
0 1
0 1
1 0
1 0
2 0
0 1
0 1
0 1
0 2
0 1
1 2
0 1
-1 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1


0 1
1 -1
2 3
1 0
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
1 0
4 5
0 1
1 0
0 1
0 1
-1 0
0 1
1 2
0 1
1 -1
0 1
1 2
0 1
1 0
0 1
0 1
3 -1
0 1
0 1
0 1
0 2
1 2
0 1
0 1
0 1
1 -1
0 1
0 1
1 2
1 3
0 1
0 1
1 -1
0 1
1 0
0 1
0 1
0 1
0 1
0 1
2 3
0 1
1 2
0 1
1 -1
0 1
0 1
1 2
1 0
0 1
3 4
0 1
0 1
0 1
1 2
0 1
0 1
0 1
0 1
0 1
1 0
0 1
0 1
0 1
-1 0
2 -1
1 0
0 1
0 1
2 -1
1 0
0 1
0 1
0 1
0 1
0 1
3 4
0 1
0 1
1 0
0 1
0 1
0 1
2 -1
0 1
1 2
0 1
0 1
0 1
0 1
0 1
7 8
0 1
2 3
0 1
0 1
1 0
1 2
0 1
0 1
0 1
1 0
-1 0
0 1
1 0
0 1
0 1
0 1
0 1
1 0
2 -1
0 1
1 2
0 1
0 2
0 1
4 -1
0 1
1 0
0 1
0 1
1 -1
1 2
1 -1
0 1
0 1
1 2
1 2
1 2
0 1
0 1
0 1
0 1
0 1
1 2
0 1
-1 0
0 1
-1 1
0 1
0 1
1 2
0 1
2 0
2 -1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
1 -1
0 1
0 1
1 0
1 0
0 1
0 1
0 1
0 1
1 0
0 1
2 -1
0 1
0 1
0 1
2 -1
0 1
0 1
1 0
0 1
0 1
1 -1
2 -1
0 1
1 0
3 4
0 1
0 1
0 1
0 1
0 1
0 1
0 1
1 2
0 1
0 1
1 2
1 0
0 1
1 0
0 1
0 1
-1 0
0 1
0 1
3 2
0 1
0 1
0 1
0 1
0 1
1 0
0 1
0 1
0 1
0 1
0 1
2 3
0 1
0 1
0 1
0 1
1 2
0 1
0 1
0 1
1 2
0 1
0 1
0 1
0 1

0 1
1 0
1 0
1 0
1 0
3 2
3 2
5 1
1 2
1 2
1 0
1 0
1 0
-1 2
3 2
1 0
3 2
1 2
3 2
3 2
3 2
1 0
1 0
0 1
0 1
1 2
2 0
1 2
1 2
1 0
3 2
1 0
1 0
1 0
1 0
1 0
2 1
1 0
1 0
-1 1
2 1
1 2
2 1
1 0
0 1
0 1
0 1
0 1
1 0
0 1
1 0
0 1
1 0
0 1
2 0
0 1
0 1
0 1
0 1
1 -1
-1 1
0 1
0 1
0 1
2 -1
1 0
0 1
0 1
0 1
0 1
0 3
1 2
0 1
0 1
2 3
3 4
0 1
0 1
0 1
0 1
0 1
1 0
1 0
0 1
0 1
0 1
0 6
0 1
0 1
0 1
0 1
1 2
1 2
1 -1
0 3
0 1
1 0
0 1
0 1
0 1
1 0
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
-1 1
2 -1
1 2
3 2
3 -1
0 1
0 1
0 1
0 1
0 1
0 1
2 3
0 1
0 1
0 1
0 1
1 -1
0 1
1 0
2 -1
0 1
2 1
0 1
0 1
0 1
1 2
2 1
-1 0
1 0
0 1
0 1
0 1
0 1
3 4
2 3
0 1
0 1
0 1
0 1
0 1
3 -1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
1 2
0 1
2 -1
0 1
0 1
2 1
1 0
0 1
0 2
1 0
1 2
0 1
0 1
0 1
0 1
1 0
0 1
-1 5
0 1
1 0
0 1
0 1
0 1
0 1
0 1
2 -1
1 -1
1 0
2 3
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
2 3
0 1
0 1
0 1
-1 6
0 1
0 1
0 1
1 2
0 1
0 1
0 1
0 1
0 1
0 1
1 0
0 1
0 1
0 1
2 -1
0 1
1 0
0 1
0 1
-1 0
0 1
0 1
1 0
0 1
1 0
1 0
1 0
1 0
1 0
0 1
0 1
0 1
0 1
0 1
0 1
2 -1
1 2
0 1
1 2
0 1

1 0
0 1
0 1
0 1
0 1
1 -1
0 1
0 1
1 2
0 1
0 1
0 1
0 1
0 1
0 1
0 1
1 2
1 2
1 0
0 1
0 1
0 1
0 1
0 1
0 1
0 1
1 2
0 1
1 0
2 3
0 1
0 1
0 1
1 2
1 2
1 0
0 1
1 -1
0 1
2 4
1 -1
0 1
0 1
1 2
0 1
2 3
1 2
0 1
-1 0
0 1
0 1
0 1
2 3
0 1
0 1
0 1
1 -1
-1 1
0 1
0 1
4 5
0 1
0 1
0 1
0 1
-1 1
0 1
0 1
0 1
1 0
2 3
1 -1
0 1
1 0
2 -1
0 1
1 2
0 1
0 1
2 3
0 1
2 -1
0 1
2 -1
0 1
0 1
0 1
0 1
0 1
4 -1
3 -1
0 1
1 2
1 0
0 1
0 1
0 1
0 1
0 1
1 2
3 2
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
3 4
1 0
0 1
0 1
1 -1
0 1
1 0
0 1
0 1
0 1
0 1
0 1
2 -1
0 1
0 1
0 1
0 1
1 0
-1 0
0 1
1 0
0 1
0 1
0 1
0 1
0 1
1 0
1 2
0 1
1 2
0 1
0 1
0 1
1 2
1 0
1 0
0 1
1 0
1 0
2 -1
0 1
0 1
0 1
1 -1
1 -1
1 0
1 0
1 2
0 1
0 1
0 1
1 0
0 1
-1 0
1 2
0 1
0 1
0 1
1 0
0 1
0 1
0 1
0 1
1 0
1 2
-1 1
0 1
0 1
0 1
0 1
0 1
-1 0
1 -1
0 1
0 1
2 -1
0 1
1 2
1 -1
1 2
0 1
1 2
0 1
0 1
0 1
0 1
-1 0
3 2
1 2
1 0
2 1
1 0
0 1
0 1
1 0
0 1
0 1
0 1
0 1
0 1
0 1
2 3
0 1
3 2
1 2
1 0
0 1
2 3
0 1
1 0
-1 2
-1 1
0 1
3 -1
0 1
0 1
1 0
1 0
1 0
0 1
2 -1
0 1
0 1
1 0
1 2
2 -1
0 1
0 1
0 1
0 1
2 1
1 2
1 0
0

0 1
1 -1
0 1
0 1
0 1
0 1
-1 1
0 1
0 1
0 1
0 1
0 1
1 0
0 1
1 0
0 1
0 1
0 1
1 2
0 1
2 -1
0 1
0 1
0 1
0 1
2 -1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
1 2
0 1
0 1
0 1
0 1
0 1
0 2
0 1
-1 1
1 -1
1 0
1 2
0 1
0 1
0 1
0 1
-1 1
0 1
3 1
1 -1
0 1
0 1
0 1
0 1
0 1
0 1
1 -1
0 1
0 1
0 1
1 0
0 1
1 2
0 1
0 1
-1 1
0 1
2 -1
0 1
1 2
0 1
-1 0
1 2
0 1
0 1
0 1
0 1
0 1
0 1
1 0
1 0
0 1
0 1
0 1
0 1
1 2
0 1
0 1
5 -1
0 1
0 1
0 1
0 1
0 1
1 -1
1 2
3 2
0 1
1 2
0 1
0 1
0 1
0 1
0 1
1 -1
1 2
0 1
0 1
0 1
1 -1
0 1
0 1
2 -1
0 1
0 1
0 1
0 1
0 1
0 1
2 3
2 -1
1 0
0 1
0 1
0 1
0 1
0 1
0 1
0 1
2 -1
1 2
0 1
2 -1
0 1
4 2
0 1
0 1
-1 1
3 -1
0 1
-1 0
0 1
0 2
0 1
0 1
1 -1
0 1
1 -1
1 2
1 3
0 1
0 1
1 2
1 -1
3 4
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
1 4
0 1
0 1
1 2
0 1
0 1
0 1
0 1
0 1
0 1
1 2
0 1
1 2
0 1
0 1
0 1
0 1
0 1
0 1
0 1
-1 0
-1 1
2 -1
1 0
2 3
0 1
0 1
1 0
0 1
1 0
0 1
0 1
1 2
0 1
0 1
0 1
0 1
0 1
0 1
0 1
7 8
0 1
0 1
0 1
0 1
1 0
0 1
1 -1
1 2
0 1
0 1
0 1
0 1
0 1
2 -1
0 1
1 2
0 1
0 1
1 0
0 1
0 2
0 1
0 1
0 1
0 1
1 0
0 1
0 1
1 -1
0 1
1 0
0 1


In [0]:
data

,ID_W,ID_H,STATUS,SITE,A3,A3_3,NFM,NFM1_O,NFM1_N,B1_1,...,H49_2_2,H49_2_3,H49_2_4,H67,H68_1B,H68_2B,H68_3B,Father,Mother,Childs
0,5,1001,3,1,1,NaN,2,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
1,5,1002,3,1,2,NaN,4,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
2,5,1003,3,1,3,NaN,1,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
3,5,1004,3,1,4,NaN,1,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
4,5,1005,3,1,5,NaN,2,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131510,27,14215911,3,189,15911,2.0,4,NaN,4.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
131511,27,14215933,3,189,15933,2.0,2,NaN,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
131512,27,14216025,3,189,16025,2.0,2,NaN,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58379.0,58378.0,1.0
131513,27,14216456,3,150,16456,2.0,2,NaN,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0


In [0]:
data.to_csv("tryInc3.csv", index=False)

In [0]:
data.shape


(131515, 355)